## A tour of MLJ

### Models, machines, basic training and testing

Let's load data and define train and test rows:

In [38]:
using MLJ
using DataFrames, Statistics

Xraw = rand(300,3)
y = exp(Xraw[:,1] - Xraw[:,2] - 2Xraw[:,3] + 0.1*rand(300))
X = DataFrame(Xraw)

train, test = partition(eachindex(y), 0.70); # 70:30 split

A *model* is a container for hyperparameters:

In [39]:
knn_model=KNNRegressor(K=10)

# KNNRegressor{Float64} @ 1…35: 
target_type             =>   Float64
K                       =>   10
metric                  =>   euclidean (generic function with 1 method)
kernel                  =>   reciprocal (generic function with 1 method)



Wrapping the model in data creates a *machine* which will store training outcomes (called *fit-results*):

In [40]:
knn = machine(knn_model, X, y)

# Machine{KNNRegressor{Float64}} @ 6…00: 
model                   =>   KNNRegressor{Float64} @ 1…35
fitresult               =>   (undefined)
cache                   =>   (undefined)
args                    =>   (omitted Tuple{DataFrame,Array{Float64,1}} of length 2)
report                  =>   empty Dict{Symbol,Any}
rows                    =>   (undefined)



Training on the training rows and evaluating on the test rows:

In [41]:
fit!(knn, rows=train)
yhat = predict(knn, X[test,:])
rms(y[test], yhat)

┌ Info: Training Machine{KNNRegressor{Float64}} @ 6…00.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69


0.06159653428003336

Or, we could have skipped the train/test definitions and evaluated one line:

In [42]:
evaluate!(knn, resampling=Holdout(fraction_train=0.7), measure=rms)

0.06159653428003336

Our machine/model constructions and associateed fit/predict syntax anticipates a powerful extension for building networks of learners described later. Changing a hyperparameter and re-evaluating:

In [43]:
knn_model.K = 20
evaluate!(knn, resampling=Holdout(fraction_train=0.7))  # `default_measure(knn) == rms` so `measure` kwarg can be dropped

0.07039579507048184

### Homogeneous ensembles

Here's a bagged ensemble model for 20 K-nearest neighbour regressors:

In [44]:
ensemble_model = EnsembleModel(atom=knn_model, n=20)

# DeterministicEnsembleModel @ 1…37: 
atom                    =>   KNNRegressor{Float64} @ 1…35
weights                 =>   0-element Array{Float64,1}
bagging_fraction        =>   0.8
rng_seed                =>   0
n                       =>   20
parallel                =>   true



In [45]:
@more

# DeterministicEnsembleModel @ 1…37: 
atom                    =>   KNNRegressor{Float64} @ 1…35
weights                 =>   0-element Array{Float64,1}
bagging_fraction        =>   0.8
rng_seed                =>   0
n                       =>   20
parallel                =>   true

## KNNRegressor{Float64} @ 1…35: 
target_type             =>   Float64
K                       =>   20
metric                  =>   euclidean (generic function with 1 method)
kernel                  =>   reciprocal (generic function with 1 method)



It can be trained and tested the same as any other model:

In [46]:
ensemble = machine(ensemble_model, X, y)
estimates = evaluate!(ensemble, resampling=CV())

Cross-validating: 100%[=========================] Time: 0:00:00


6-element Array{Float64,1}:
 0.1347375467238205 
 0.08720682525943388
 0.08190452711576283
 0.17726314060081041
 0.08240531838109268
 0.0602395423402537 

In [47]:
(mean(estimates), std(estimates))

(0.10395948340352901, 0.043505869186543776)

### Systematic tuning

Let's simultaneously tune the ensemble's `bagging_fraction` and the K-nearest neighbour hyperparameter `K`. Since one of these models is a field of the other, we have nested hyperparameters:

In [48]:
params(ensemble_model)

Params(:atom => Params(:target_type => Float64, :K => 20, :metric => MLJ.KNN.euclidean, :kernel => MLJ.KNN.reciprocal), :weights => Float64[], :bagging_fraction => 0.8, :rng_seed => 0, :n => 20, :parallel => true)

To define a tuning grid, we construct ranges for the two parameters and collate these ranges following the same pattern above (omitting parameters that don't change):

In [49]:
B_range = range(ensemble_model, :bagging_fraction, lower= 0.5, upper=1.0, scale = :linear)
K_range = range(knn_model, :K, lower=1, upper=100, scale=:log10)
nested_ranges = Params(:atom => Params(:K => K_range), :bagging_fraction => B_range)

Params(:atom => Params(:K => NumericRange @ 6…22), :bagging_fraction => NumericRange @ 1…98)

Now we choose a tuning strategy:

In [50]:
tuning = Grid(resolution=12)

# Grid @ 6…06: 
resolution              =>   12
parallel                =>   true



And a resampling strategy:

In [51]:
resampling = Holdout(fraction_train=0.8)

# Holdout @ 2…71: 
fraction_train          =>   0.8



And define a new model which wraps the these strategies around our ensemble model:

In [52]:
tuned_ensemble_model = TunedModel(model=ensemble_model, 
    tuning=tuning, resampling=resampling, nested_ranges=nested_ranges)

# DeterministicTunedModel @ 4…26: 
model                   =>   DeterministicEnsembleModel @ 1…37
tuning                  =>   Grid @ 6…06
resampling              =>   Holdout @ 2…71
measure                 =>   nothing
operation               =>   predict (generic function with 19 methods)
nested_ranges           =>   Params(:atom => Params(:K => NumericRange @ 6…22), :bagging_fraction => NumericRange @ 1…98)
report_measurements     =>   true



Fitting the corresponding machine tunes the underlying model (in this case an ensemble) and retrains on all supplied data:

In [53]:
tuned_ensemble = machine(tuned_ensemble_model, X[train,:], y[train])
fit!(tuned_ensemble);

┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 3…55.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69
Searching a 132-point grid for best model: 100%[=========================] Time: 0:00:06
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:130


We can inspect the best model by looking at the machine's `report` field (every machine has one):

In [54]:
tuned_ensemble.report

Dict{Symbol,Any} with 4 entries:
  :measurements     => [0.141569, 0.166763, 0.176318, 0.177218, 0.193808, 0.220…
  :models           => DeterministicEnsembleModel{Tuple{Array{Float64,2},Array{…
  :best_model       => DeterministicEnsembleModel @ 8…30
  :best_measurement => 0.12144

In [55]:
best_model = tuned_ensemble.report[:best_model]
@show best_model.bagging_fraction
@show best_model.atom.K

best_model.bagging_fraction = 0.7272727272727273
(best_model.atom).K = 1


1

Evaluating the tuned model:

In [56]:
yhat = predict(tuned_ensemble, X[test,:])
rms(yhat, y[test])

0.0565539515409894

Or, using all the data, get cross-validation estimates, with re-tuning on each fold complement (nested resampling):

In [57]:
tuned_ensemble = machine(tuned_ensemble_model, X, y)
evaluate!(tuned_ensemble, resampling=CV(nfolds=4), verbosity=2)

Cross-validating:  20%[=====>                   ]  ETA: 0:00:00┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…27.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69
Searching a 132-point grid for best model: 100%[=========================] Time: 0:00:06
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:130
Cross-validating:  40%[==========>              ]  ETA: 0:00:10┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…27.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69
Searching a 132-point grid for best model: 100%[=========================] Time: 0:00:06
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:130
Cross-validating:  60%[===============>         ]  ETA: 0:00:08┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…27.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69
Searching a 132-point grid for best model: 100%[==

4-element Array{Float64,1}:
 0.10123630662600994
 0.06952722322140588
 0.10693127102910563
 0.06679536114506222

### Learning networks

MLJ has a flexible interface for building networks from multiple machine learning elements, whose complexity extend beyond linear "pipelines", and with a minimal of added abstraction.

In MLJ, a *learning network* is a graph whose nodes apply an operation, such as `predict` or `transform`, using a fixed machine (requiring training) - or which, alternatively, applies a regular (untrained) mathematical operation to its input(s). In practice, a learning network works with *fixed* sources for its training/evaluation data, but can be built and tested in stages. By contrast, an *exported learning network* is a learning network exported as a stand-alone, re-usable `Model` object, to which all the MLJ `Model`  meta-algorthims can be applied (ensembling, systematic tuning, etc). 

As we shall see, exporting a learning network as a reusable model, is very easy. 

### Building a simple learning network

![](wrapped_ridge.png)

The diagram above depicts a learning network which standardises the input data, `X`, learns an optimal Box-Cox transformation for the target, `y`, predicts new targets using ridge regression, and then inverse-transforms those predictions (for later comparison with the original test data). The machines are labelled yellow. 

To implement the network, we begin by loading all data needed for training and evaluation into *source nodes*:

In [58]:
Xs = source(X)
ys = source(y)

Source @ 1…79

We label nodes according to their outputs in the diagram. Notice that the nodes `z` and `yhat` use the same machine `box` for different operations. 

To construct the `W` node we first need to define the machine `stand` that it will use to transform inputs. 

In [59]:
stand_model = Standardizer()
stand = machine(stand_model, Xs)

NodalMachine @ 1…22 = machine(Standardizer @ 2…11, 7…44)

Because `Xs` is a node, instead of concrete data, we can call `transform` on the machine without first training it, and the result is the new node `W`, instead of concrete transformed data:

In [60]:
W = transform(stand, Xs)

Node @ 5…36 = transform(1…22, 7…44)

To get actual transformed data we *call* the node appropriately, which will require we first train the node. Training a node, rather than a machine, triggers training of *all* necessary machines in the network.

In [61]:
fit!(W, rows=train)
W()          # transform all data
W(rows=test) # transform only test data
W(X[3:4,:])  # transform any data, new or old

┌ Info: Training NodalMachine{Standardizer} @ 1…22.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69


(x1 = [-1.3299, 0.515097], x2 = [-0.475242, 0.106812], x3 = [0.0650451, 1.18952])

If you like, you can think of `W` (and the other nodes we will define) as "dynamic data": `W` is *data*, in the sense that  it an be called ("indexed") on rows, but *dynamic*, in the sense the result depends on the outcome of training events. 

The other nodes of our network are defined similarly:

In [62]:
box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
box = machine(box_model, ys)
z = transform(box, ys)

ridge_model = RidgeRegressor(lambda=0.1)
ridge =machine(ridge_model, W, z)
zhat = predict(ridge, W)

yhat = inverse_transform(box, zhat)

Node @ 1…58 = inverse_transform(7…73, predict(1…61, transform(1…22, 7…44)))

We are ready to train and evaluate the completed network. Notice that the standardizer, `stand`, is *not* retrained, as MLJ remembers that it was trained earlier:

In [63]:
fit!(yhat, rows=train)
rms(y[test], yhat(rows=test)) # evaluate

┌ Info: Not retraining NodalMachine{Standardizer} @ 1…22. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Training NodalMachine{UnivariateBoxCoxTransfor…} @ 7…73.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69
┌ Info: Training NodalMachine{RidgeRegressor{Float64}} @ 1…61.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69


0.02934991155459362

In [64]:
yhat(X[3:4,:])  # predict on new or old data

2-element Array{Float64,1}:
 0.3034576112425516
 0.228688932808509 

We can change hyperparameters and retrain:

In [65]:
ridge_model.lambda = 0.01
fit!(yhat, rows=train) 
rms(y[test], yhat(rows=test))

┌ Info: Not retraining NodalMachine{UnivariateBoxCoxTransfor…} @ 7…73. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Not retraining NodalMachine{Standardizer} @ 1…22. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Updating NodalMachine{RidgeRegressor{Float64}} @ 1…61.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:73


0.029261428560472036

> **Notable feature.** The machine, `ridge::NodalMachine{RidgeRegressor}`, is retrained, because its underlying model has been mutated. However, since the outcome of this training has no effect on the training inputs of the machines `stand` and `box`, these transformers are left untouched. (During construction, each node and machine in a learning network determines and records all machines on which it depends.) This behaviour, which extends to exported learning networks, means we can tune our wrapped regressor without re-computing transformations each time the hyperparameter is changed. 

### Exporting a learning network as a composite model

To export a learning network:
- Define a new `mutable struct` model type.
- Wrap the learning network code in a model `fit` method.

All learning networks that make determinisic (or, probabilistic) predictions export as models of subtype `Deterministic{Node}` (respectively, `Probabilistic{Node}`):


In [66]:
mutable struct WrappedRidge <: Deterministic{Node}
    ridge_model
end

Now satisfied that our wrapped Ridge Regression learning network works, we simply cut and paste its defining code into a `fit` method: 

In [67]:
function MLJ.fit(model::WrappedRidge, X, y)
    Xs = source(X)
    ys = source(y)

    stand_model = Standardizer()
    stand = machine(stand_model, Xs)
    W = transform(stand, Xs)

    box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
    box = machine(box_model, ys)
    z = transform(box, ys)

    ridge_model = model.ridge_model ###
    ridge =machine(ridge_model, W, z)
    zhat = predict(ridge, W)

    yhat = inverse_transform(box, zhat)
    fit!(yhat, verbosity=0)
    
    return yhat
end

The line marked `###`, where the new exported model's hyperparameter `ridge_model` is spliced into the network, is the only modification.

This completes the export process.

> **What's going on here?** MLJ's machine interface is built atop a more primitive *[model](adding_new_models.md)* interface, implemented for each algorithm. Each supervised model type (eg, `RidgeRegressor`) requires model `fit` and `predict` methods, which are called by the corresponding machine `fit!` and `predict` methods. We don't need to define a  model `predict` method here because MLJ provides a fallback which simply calls the node returned by `fit` on the data supplied: `MLJ.predict(model::Supervised{Node}, Xnew) = yhat(Xnew)`.

Let's now let's wrap our composite model as a tuned model and evaluate on the Boston dataset:

In [68]:
X, y = X_and_y(load_boston())
train, test = partition(eachindex(y), 0.7)
wrapped_model = WrappedRidge(ridge_model)

# WrappedRidge @ 1…13: 
ridge_model             =>   RidgeRegressor{Float64} @ 1…30



In [69]:
params(wrapped_model)

Params(:ridge_model => Params(:target_type => Float64, :lambda => 0.01))

In [70]:
nested_ranges = Params(:ridge_model => Params(:lambda => range(ridge_model, :lambda, lower=0.1, upper=100.0, scale=:log10)))

Params(:ridge_model => Params(:lambda => NumericRange @ 1…57))

In [71]:
tuned_wrapped_model = TunedModel(model=wrapped_model, tuning=Grid(resolution=20),
resampling=CV(), measure=rms, nested_ranges=nested_ranges);

In [72]:
tuned_wrapped = machine(tuned_wrapped_model, X, y)
evaluate!(tuned_wrapped, resampling=Holdout(fraction_train=0.7), measure=rms, verbosity=0) |> mean  # nested resampling estimate

Searching a 20-point grid for best model:   5%[=>                       ]  ETA: 0:00:00

6.88236977264247